# Complex CAPTCHA-Solver Demo 
# (Note: using data generated by 'captcha' python library)
---

In [ ]:
# Functions from other notebook file.
from ipynb.fs.full.shared_functions import *

import tensorflow
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as pyplot
import os

In [ ]:
# Suppress tensorflow log messages.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
# GLOBALS
DATA_DIRECTORY = os.path.join(os.getcwd() + '/datasets/complex')

IMAGE_HEIGHT = 100
IMAGE_WIDTH = 100
IMAGE_CHANNELS = 3

CATEGORIES = 10 # represents digits 0-9
DIMENSIONS = 4  # 4-digit CAPTCHA images

TRAINING_EPOCHS = 20

TRAINING_BATCH_SIZE = 64
VALIDATION_BATCH_SIZE = 64
TESTING_BATCH_SIZE = 128

---
## Prepare the Dataset



Store each CAPTCHA-image file path with its respective label
into a pandas DataFrame.

In [ ]:
data_frame = create_captcha_dataframe(DATA_DIRECTORY)
data_frame.head()

---
Shuffle the data and create a training set, validation set, and testing set.

In [ ]:
train_indices, validation_indices, test_indices = shuffle_and_split_data(data_frame)

print('train count: %s, validation count: %s, test count: %s' % (
    len(train_indices), len(validation_indices), len(test_indices)))

---
## Neural Network Structure

**'relu'** stands for **'Rectified Linear Unit'**, the most commonly used activation function for convolutional neural networks.

**'softmax'** is another activation function used for classifying data.

Activation functions are analagous to the 'firing' of neurons in biological neural networks.

**Layers**:
- Convolutional layer: applies a filter to the CAPTCHA image to extract features (characters and/or digits) from the image
- Pooling layer: immediately follows a convolutional layer and used to downscale the image after each filter is applied
- Flattening layer: converts the CAPTCHA image represented as a 3D tensor (array) to a 1D tensor
- Dense layer: used to assist with operations on an n-dimensional tensor such as rotation, scaling, etc
- Reshape layer: used to restructure the output of the neural network

In [ ]:
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Conv2D, MaxPool2D, GlobalMaxPool2D, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Model

input_layer = tensorflow.keras.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS))

hidden_layers = layers.Conv2D(32, 3, activation='relu')(input_layer)
hidden_layers = layers.MaxPooling2D((2, 2))(hidden_layers)
hidden_layers = layers.Conv2D(64, 3, activation='relu')(hidden_layers)
hidden_layers = layers.MaxPooling2D((2, 2))(hidden_layers)
hidden_layers = layers.Conv2D(64, 3, activation='relu')(hidden_layers)
hidden_layers = layers.MaxPooling2D((2, 2))(hidden_layers)

hidden_layers = layers.Flatten()(hidden_layers)

hidden_layers = layers.Dense(1024, activation='relu')(hidden_layers)
hidden_layers = layers.Dense(DIMENSIONS * CATEGORIES, activation='softmax')(hidden_layers)
hidden_layers = layers.Reshape((DIMENSIONS, CATEGORIES))(hidden_layers)

model = models.Model(inputs=input_layer, outputs=hidden_layers)

model.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics= ['accuracy'])
model.summary()

---
## Train the Model

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint


training_set_generator = get_captcha_generator(data_frame, train_indices, \
                                               for_training=True, batch_size=TRAINING_BATCH_SIZE)
validation_set_generator = get_captcha_generator(data_frame, validation_indices, \
                                                 for_training=True, batch_size=VALIDATION_BATCH_SIZE)

callbacks = [
    ModelCheckpoint("./model_checkpoint", monitor='val_loss')
]

history = model.fit(training_set_generator,
                    steps_per_epoch=len(train_indices)//TRAINING_BATCH_SIZE,
                    epochs=TRAINING_EPOCHS,
                    validation_data=validation_set_generator,
                    validation_steps=len(validation_indices)//VALIDATION_BATCH_SIZE)

---
## Analyze Model Performance

Plot the accuracy and loss metrics

In [ ]:
figure, axes = pyplot.subplots(1, 2, figsize=(20, 5))

axes[0].plot(history.history['accuracy'], label='Training accuracy')
axes[0].plot(history.history['val_accuracy'], label='Validation accuracy')
axes[0].set_xlabel('Epochs')
axes[0].legend() 

axes[1].plot(history.history['loss'], label='Training loss')
axes[1].plot(history.history['val_loss'], label='Validation loss')
axes[1].set_xlabel('Epochs')
axes[1].legend()

Use some 'unseen' test samples

In [ ]:
testing_set_generator = get_captcha_generator(data_frame, test_indices, 
                                              for_training=False, batch_size=TESTING_BATCH_SIZE)
dict(zip(model.metrics_names, model.evaluate(testing_set_generator, steps=len(test_indices)//TESTING_BATCH_SIZE)))

---
## Visualize Model Performance

In [ ]:
testing_set_generator = get_captcha_generator(data_frame, test_indices, 
                                              for_training=False, batch_size=TESTING_BATCH_SIZE)

captcha_images, captcha_text = next(testing_set_generator)

predictions = model.predict_on_batch(captcha_images)

true_values = tensorflow.math.argmax(captcha_text, axis=-1)
predictions = tensorflow.math.argmax(predictions, axis=-1)

In [ ]:
display_predictions_from_model(captcha_images, predictions, true_values)

## Save the Model

In [ ]:
#model.save('my_model')